In [71]:
import os
import numpy as np
import nibabel as nib
from dipy.io.image import load_nifti, save_nifti
from dipy.io.gradients import read_bvals_bvecs
from dipy.core.gradients import gradient_table
from dipy.segment.mask import median_otsu
from dipy.align.imaffine import MutualInformationMetric, AffineRegistration
from dipy.align.transforms import RigidTransform3D
from dipy.viz import regtools


import matplotlib.pyplot as plt

from nibabel.processing import resample_from_to
from nilearn import plotting

from dipy.align.imaffine import AffineRegistration
from dipy.align.transforms import RigidTransform3D
from dipy.align.metrics import SSDMetric
from dipy.align.imaffine import AffineRegistration, MutualInformationMetric

from dipy.align import motion_correction
from dipy.align.reslice import reslice

from dipy.align.transforms import (TranslationTransform3D,
                                   RigidTransform3D,
                                   AffineTransform3D)
from dipy.align import affine_registration, register_dwi_to_template



In [66]:
main_dir = "/media/ist/Drive2/MANSOOR/Neuroimaging-Project/DTI-Analysis"
data_dir = f"{main_dir}/Data/test/test-sub-01-pre"
save_prep_dir = f"{data_dir}/prep-dti"
results_dir = f"{main_dir}/results"
static_img_file = f"{data_dir}/target.nii.gz"

filename = "sub_01_pre"

In [41]:

# load the DTI data
img_file = f'{data_dir}/{filename}.nii.gz'
bvals_file = f'{data_dir}/{filename}.bval'
bvecs_file = f'{data_dir}/{filename}.bvec'

data, affine = load_nifti(img_file)
bvals, bvecs = read_bvals_bvecs(bvals_file, bvecs_file)
gtab = gradient_table(bvals, bvecs)


Skull stripping - brain segmentation

In [39]:

# Step 1: Skull stripping using median otsu method
# Assuming 'data' is the 4D DTI dataset and we want to use the first volume for skull stripping
# Here '0' is the index of the b0 volume which is typically used for skull stripping
data, mask = median_otsu(data, vol_idx=[0], median_radius=2, numpass=1)

# Save skull-stripped data and mask
ss_data_img = nib.Nifti1Image(data, affine)
ss_mask_img = nib.Nifti1Image(mask.astype(np.uint8), affine)
save_nifti(f'{save_prep_dir}/{filename}-skull_stripped.nii.gz', data, affine)
save_nifti(f'{save_prep_dir}/{filename}-brain_mask.nii.gz', mask.astype(np.uint8), affine)



Motion correction (across DTI volumes)

In [78]:
##### testing the script for a few volumes  #####
data_small = data[..., :3]
bvals_small = bvals[:3]
bvecs_small = bvecs[:3]
gtab = gradient_table(bvals_small, bvecs_small)

# data_corrected, reg_affines = motion_correction(data_small, gtab, affine)

# Save the motion-corrected data
# save_nifti(f'{save_prep_dir}/{filename}-motion_corrected.nii.gz', data_corrected.get_fdata(), data_corrected.affine)


In [ ]:
# Apply motion correction
gtab = gradient_table(bvals, bvecs)

corrected_data, transformed_affines = motion_correction(data, gtab, affine)

# Save the motion-corrected data
save_nifti(f'{save_prep_dir}/{filename}-motion_corrected.nii.gz', corrected_data.get_fdata(), transformed_affines.affine)

print("Motion correction complete. Corrected data and gradient vectors saved.")


Registration to a standard space (e.g., MNI template)

In [80]:
# Registration to standard space using an affine transform
standard, standard_affine = load_nifti(static_img_file)

# Configure the registration framework
affreg = AffineRegistration(metric=MutualInformationMetric(),
                            level_iters=[10000, 1000, 100, 10],
                            sigmas=[3.0, 1.0, 0.1, 0.0],
                            factors=[4, 2, 1, 0.25])

# Start with a translation transform
transform = TranslationTransform3D()
params0 = None
starting_affine = affreg.optimize(static=standard, moving=data[..., gtab.b0s_mask].mean(axis=3),
                                  transform=transform, params0=params0,
                                  static_grid2world=standard_affine,
                                  moving_grid2world=affine)

# Then a rigid body transform (rotation and translation)
transform = RigidTransform3D()
rigid_affine = affreg.optimize(static=standard, moving=data[..., gtab.b0s_mask].mean(axis=3),
                               transform=transform, params0=starting_affine.affine,
                               static_grid2world=standard_affine,
                               moving_grid2world=affine)

# Finally, a full affine transform (rotation, translation, scaling, and shearing)
transform = AffineTransform3D()
affine_map = affreg.optimize(static=standard, moving=data[..., gtab.b0s_mask].mean(axis=3),
                             transform=transform, params0=rigid_affine.affine,
                             static_grid2world=standard_affine,
                             moving_grid2world=affine)

# Apply transformation to the whole DTI dataset
registered_data = np.zeros_like(data)
for i in range(data.shape[-1]):
    registered_data[..., i] = affine_map.transform(data[..., i])

# Save the registered image
save_nifti(f'{save_prep_dir}/{filename}-registered_to_standard.nii.gz', registered_data, standard_affine)

print("DTI preprocessing complete. Data is registered to standard space.")
print("DTI preprocessing complete.")


Optimizing level 3 [max iter: 10000]
Optimizing level 2 [max iter: 1000]
Optimizing level 1 [max iter: 100]
Optimizing level 0 [max iter: 10]
Optimizing level 3 [max iter: 10000]


ValueError: 'x0' must only have one dimension.